In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer
#from nltk.metrics import edit_distance
from nltk.stem.porter import *
stemmer = PorterStemmer()
#from nltk.stem.snowball import SnowballStemmer #0.003 improvement but takes twice as long as PorterStemmer
#stemmer = SnowballStemmer('english')
import re
#import enchant
import random
random.seed(2016)
from autocorrect import spell
import enchant



## Load data

In [37]:
df_train = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/test.csv', encoding="ISO-8859-1") #update here
df_prod = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/product_descriptions.csv') #update here
df_attr = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/attributes.csv')



In [38]:
##TEMPORARY FOR TESTING
df_train = df_train[df_train.product_uid <100999 ]
df_test = df_test[df_test.product_uid <100999 ]
df_prod = df_prod[df_prod.product_uid <100999 ]
df_attr = df_attr[df_attr.product_uid <100999 ]


In [39]:
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})

## Pre processing 

In [54]:
import string
from nltk.corpus import stopwords
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}

f = open('spelling.txt','r')
zspell = {}
for line in f:
    a, b = line.strip("\n").split("|")
    zspell[a]=b
f.close()

def text_process(s):
    """
    Takes in a string of text, then performs the following:
    1. Clean up data
    2. Remove all stopwords
    3. Returns a list of the cleaned text  
    
    """
    
    s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
    s = re.sub(r"([a-z])([A-Z])", r"\1 \2", s) #Split words with a A
    s = s.lower()
    s = s.replace("  "," ")
    s = re.sub(r"([0-9])( *),( *)([0-9])", r"\1\4", s)
    s = s.replace(","," ")
    s = s.replace("$"," ")
    s = s.replace("?"," ")
    s = s.replace("-"," ")
    s = s.replace("//","/")
    s = s.replace("..",".")
    s = s.replace(" / "," ")
    s = s.replace(" \\ "," ")
    s = s.replace("."," . ")
    s = s.replace("   "," ")
    s = s.replace("  "," ").strip(" ")
    s = re.sub(r"(.*)\.$", r"\1", s) #end period
    s = re.sub(r"(.*)\/$", r"\1", s) #end period
    s = re.sub(r"^\.(.*)", r"\1", s) #start period
    s = re.sub(r"^\/(.*)", r"\1", s) #start slash
    s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
    s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
    s = s.replace(" x "," xbi ")
    s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
    s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
    s = s.replace("*"," xbi ")
    s = s.replace(" by "," xbi ")
    s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
    s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
    s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
    s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
    s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
    s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
    s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
    s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
    s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
    s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
    s = s.replace("°"," degrees ")
    s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
    s = s.replace(" v "," volts ")
    s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
    s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
    s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
    s = s.replace("  "," ")
    s = s.replace(" . "," ")
    
    # Check characters to see if they are in punctuation
    #nopunc = [char for char in s.split() if char not in string.punctuation]

    # Join the characters again to form the string.
    #nopunc = ''.join(nopunc)
    
    #s = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    #s = ''.join(s)
    
    s = (" ").join([z for z in s.split(" ") if z not in stopwords.words('english')])
    s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
    s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
    s = s.lower()
    s = (" ").join([str(zspell[z]) if z in zspell else z for z in s.split(" ")])
    return s
    
   

In [55]:
df_prod['product_description'] .head(5).apply(text_process)[1]

'behr premium textur deckov innov solid color coat bring old weather wood concret back life advanc 100% acryl resin formula creat durabl coat tire worn deck rejuven whole new look best result sure properli prepar surfac use applic behr product display california residents: see&nbsp;proposit 65in. format reviv wood composit deck rail porch boat dock also great concret pool deck patio sidewalk 100% acryl solid color coat resist crack peel conceal splinter crack 1/4in. provid durabl mildew resist finish cover 75sq.ft. 2 coat per gallon creat textur slip resist finish best result prepar appropri behr product wood concret surfac actual paint color may vari screen printer represent color avail tint store onlin price includ paint care fee follow states: ca co ct mn ri vt'

In [56]:
##performing cleaning and stemming
df_train['product_title'] = df_train['product_title'].apply(text_process)
#df_train['product_title'] = df_train['product_title'].apply(stem)

df_train['search_term'] = df_train['search_term'].apply(text_process)
#df_train['search_term'] = df_train['search_term'].apply(stem)

df_test['product_title'] = df_test['product_title'].apply(text_process)
#df_test['product_title'] = df_test['product_title'].apply(stem)

df_test['search_term'] = df_test['search_term'].apply(text_process)
#df_test['search_term'] = df_test['search_term'].apply(stem)

df_prod['product_description'] = df_prod['product_description'].apply(text_process)
#df_prod['product_description'] = df_prod['product_description'].apply(stem)

df_brand['brand'] = df_brand['brand'].apply(text_process)
#df_brand['brand'] = df_brand['brand'].apply(stem)

In [60]:
df_prod.head()

,product_uid,product_description
0,100001,angl make joint stronger also provid consist s...
1,100002,behr premium textur deckov innov solid color c...
2,100003,classic architectur meet contemporari design e...
3,100004,grape solar 265watt. polycrystallin pv solar p...
4,100005,updat bathroom delta vero singl handl shower f...


## Vectorization

In [103]:
from sklearn.feature_extraction.text import CountVectorizer
#messages_bow = bow_transformer.transform(messages['message'])
bow_transformer = CountVectorizer(analyzer='word').fit(df_prod['product_description'])

In [107]:
messages_bow = bow_transformer.transform(df_prod['product_description'])
print ('Shape of Sparse Matrix: ', messages_bow.shape)
print ('Amount of Non-Zero occurences: ', messages_bow.nnz)
print ('sparsity: %.2f%%' % (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1])))

Shape of Sparse Matrix:  (998, 6674)
Amount of Non-Zero occurences:  71052
sparsity: 1.07%


In [109]:
messages_bow.get_shape

<bound method csr_matrix.get_shape of <998x6674 sparse matrix of type '<class 'numpy.int64'>'
	with 71052 stored elements in Compressed Sparse Row format>>

In [4]:
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
num_train = df_train.shape[0]
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_prod, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')

In [6]:
df_all.head()

,id,product_title,product_uid,relevance,search_term,product_description,brand
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Update your bathroom with the Delta Vero Singl...,Delta
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Update your bathroom with the Delta Vero Singl...,Delta


In [12]:
df_all.shape

(75067, 7)

In [11]:
df_all[df_all.product_uid == 100001]

,id,product_title,product_uid,relevance,search_term,product_description,brand
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.0,angle bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.5,l bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
74067,1,Simpson Strong-Tie 12-Gauge Angle,100001,NaN,90 degree bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
74068,4,Simpson Strong-Tie 12-Gauge Angle,100001,NaN,metal l brackets,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
74069,5,Simpson Strong-Tie 12-Gauge Angle,100001,NaN,simpson sku able,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
74070,6,Simpson Strong-Tie 12-Gauge Angle,100001,NaN,simpson strong ties,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
74071,7,Simpson Strong-Tie 12-Gauge Angle,100001,NaN,simpson strong tie hcc668,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
74072,8,Simpson Strong-Tie 12-Gauge Angle,100001,NaN,wood connectors,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie


In [16]:
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))

KeyboardInterrupt: 